# インポート

In [6]:
# import numpy as np
# import lightgbm as lgb
import pandas as pd
from pathlib import Path
import create_population
import preprocessing
from feature_producing import FeatureCreator
from training import Trainer
from evaluation import Evaluator
%load_ext autoreload


ModuleNotFoundError: No module named 'lightgbm'

In [43]:
%autoreload

# 学習母集団の作成
期間を指定すると、学習母集団である（日付、race_id、horse_id）の組みを作る

In [8]:
COMMON_DIR = Path("..","..", "common","data")
RAWDF_DIR = COMMON_DIR / "rawdf"
race_info =pd.read_csv(RAWDF_DIR / "race_info.csv", sep = "\t")
race_info

,race_id,title,info_1,info_2
0,2.023010e+11,2歳未勝利,"['芝右1200m', '天候:晴', '芝:良', '発走:09:50']","['2023年7月22日', '1回札幌1日目', '2歳未勝利', '混', '指', '..."
1,2.023010e+11,3歳未勝利,"['ダ右1000m', '天候:晴', 'ダート:良', '発走:10:25']","['2023年7月22日', '1回札幌1日目', '3歳未勝利', '指', '馬齢']"
2,2.023010e+11,3歳未勝利,"['ダ右1700m', '天候:晴', 'ダート:良', '発走:10:55']","['2023年7月22日', '1回札幌1日目', '3歳未勝利', '指', '馬齢']"
3,2.023010e+11,3歳未勝利,"['芝右1500m', '天候:晴', '芝:良', '発走:11:25']","['2023年7月22日', '1回札幌1日目', '3歳未勝利', '牝', '指', '..."
4,2.023010e+11,2歳新馬,"['ダ右1700m', '天候:晴', 'ダート:良', '発走:12:15']","['2023年7月22日', '1回札幌1日目', '2歳新馬', '混', '指', '馬齢']"
...,...,...,...,...
3461,2.022050e+11,3歳未勝利,"['ダ左1400m', '天候:晴', 'ダート:良', '発走:10:10']","['2022年2月5日', '1回東京3日目', '3歳未勝利', '牝', '馬齢']"
3462,2.022050e+11,3歳未勝利,"['ダ左1300m', '天候:晴', 'ダート:良', '発走:10:40']","['2022年2月5日', '1回東京3日目', '3歳未勝利', '馬齢']"
3463,2.022050e+11,3歳未勝利,"['ダ左1600m', '天候:晴', 'ダート:良', '発走:11:10']","['2022年2月5日', '1回東京3日目', '3歳未勝利', '馬齢']"
3464,2.022050e+11,3歳1勝クラス,"['ダ左1400m', '天候:晴', 'ダート:良', '発走:11:40']","['2022年2月5日', '1回東京3日目', '3歳1勝クラス', '馬齢']"


In [10]:
import numpy as np

# race_info["info_2"] = race_info["info_2"].astype(str)
# race_info["date"] = pd.to_datetime(race_info["info_2"].map(lambda x: eval(x, {"nan": np.nan})[0]), format="%Y年%m月%d日")
# race_info = race_info.copy()
race_info["date"] = pd.to_datetime(race_info["info_2"].map(lambda x: eval(x, {"nan": np.nan})[0] if isinstance(eval(x, {"nan": np.nan}), list) else np.nan), format="%Y年%m月%d日")
race_info["date"]

0      2023-07-22
1      2023-07-22
2      2023-07-22
3      2023-07-22
4      2023-07-22
          ...    
3461   2022-02-05
3462   2022-02-05
3463   2022-02-05
3464   2022-02-05
3465   2022-02-05
Name: date, Length: 3466, dtype: datetime64[ns]

In [66]:
population = create_population.create(
    from_ ="2023-01-01",
    to_ = "2023-12-31",
)

In [67]:
population["race_id"]

0        202301010101
1        202301010101
2        202301010101
3        202301010101
4        202301010101
             ...     
47667    202310030812
47668    202310030812
47669    202310030812
47670    202310030812
47671    202310030812
Name: race_id, Length: 47672, dtype: int64

In [68]:
population

,race_id,date,horse_id
0,202301010101,2023-07-22,2021101429
1,202301010101,2023-07-22,2021105872
2,202301010101,2023-07-22,2021106854
3,202301010101,2023-07-22,2021105553
4,202301010101,2023-07-22,2021100648
...,...,...,...
47667,202310030812,2023-09-03,2020105644
47668,202310030812,2023-09-03,2018106584
47669,202310030812,2023-09-03,2020101781
47670,202310030812,2023-09-03,2019106647


# データ加工

In [21]:
import preprocessing
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [79]:
%autoreload

In [70]:
results = pd.read_csv(RAWDF_DIR / "results.csv", sep = "\t")
results["race_id"]

0        202301010101
1        202301010101
2        202301010101
3        202301010101
4        202301010101
             ...     
47746    202205010305
47747    202205010305
47748    202205010305
47749    202205010305
47750    202205010305
Name: race_id, Length: 47751, dtype: int64

In [48]:
# Ensure the data types are correctly set
results_preprocessd = results_preprocessd.astype({
	'race_id': 'int64',
	'horse_id': 'int64',
	'jockey_id': 'int64',
	'trainer_id': 'int64',
	'owner_id': 'int64',
	'rank': 'int32',
	'umaban': 'int32',
	'wakuban': 'int32',
	'tansyo_odds': 'float64',
	'popularity': 'int32',
	'kinryou': 'float64',
	'sex': 'int64',
	'age': 'int32',
	'weight': 'int32',
	'weight_diff': 'int32'
})

In [58]:
COMMON_DATA_DIR = Path("..","..","common","data")
RAWDF_DIR = COMMON_DATA_DIR/"rawdf"
input_dir: Path = RAWDF_DIR
results = pd.read_csv(input_dir / "results.csv", sep="\t")
# df = pd.read_csv(input_dir / "results.csv", sep="\t").query("race_id in @population['race_id']")
results["race_id"]

0        202301010101
1        202301010101
2        202301010101
3        202301010101
4        202301010101
             ...     
47746    202205010305
47747    202205010305
47748    202205010305
47749    202205010305
47750    202205010305
Name: race_id, Length: 47751, dtype: int64

In [74]:
#レース結果テーブルの前処理
results_preprocessd = preprocessing.process_results()

In [13]:
# 重複チェック
results_preprocessd.duplicated(subset=["race_id", "horse_id"]).sum()

np.int64(0)

In [76]:
# 欠損チェック
results_preprocessd.isnull().sum()

race_id        0
horse_id       0
jockey_id      0
trainer_id     0
owner_id       0
rank           0
umaban         0
wakuban        0
tansyo_odds    0
popularity     0
kinryou        0
sex            0
age            0
weight         0
weight_diff    0
dtype: int64

In [80]:
# 馬の過去成績テーブルの前処理
horse_results_preprocessd = preprocessing.process_horse_results()

In [81]:
horse_results_preprocessd

,horse_id,date,rank,prize,rank_diff,weather,race_type,course_len,ground_condition,race_class,n_horses
0,2021101429,2024-11-09,9.0,0.0,0.2,0.0,1,1400,0.0,4.0,14
1,2021101429,2024-10-06,1.0,1140.0,0.0,1.0,1,1400,0.0,3.0,13
2,2021101429,2024-09-01,2.0,625.2,0.4,0.0,1,1400,0.0,5.0,7
3,2021101429,2024-08-10,2.0,629.2,0.0,0.0,1,1400,0.0,5.0,14
4,2021101429,2024-04-06,13.0,0.0,0.9,1.0,1,1600,1.0,7.0,16
...,...,...,...,...,...,...,...,...,...,...,...
213811,2019103076,2022-01-10,8.0,0.0,0.7,1.0,1,2000,0.0,1.0,17
213812,2019103076,2021-12-12,5.0,51.0,0.2,1.0,1,2000,0.0,1.0,17
213813,2019103076,2021-11-20,12.0,0.0,2.0,0.0,1,1200,0.0,1.0,16
213814,2019103076,2021-07-18,7.0,0.0,1.1,0.0,1,1200,0.0,1.0,12


In [82]:
horse_results_preprocessd.info()

<class 'pandas.core.frame.DataFrame'>
Index: 210604 entries, 0 to 213815
Data columns (total 11 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   horse_id          210604 non-null  int64         
 1   date              210604 non-null  datetime64[ns]
 2   rank              210604 non-null  float64       
 3   prize             210604 non-null  float64       
 4   rank_diff         210344 non-null  float64       
 5   weather           210385 non-null  float64       
 6   race_type         210604 non-null  int64         
 7   course_len        210604 non-null  int32         
 8   ground_condition  210600 non-null  float64       
 9   race_class        150061 non-null  float64       
 10  n_horses          210604 non-null  int64         
dtypes: datetime64[ns](1), float64(6), int32(1), int64(3)
memory usage: 18.5 MB


In [83]:
# 欠損チェック
horse_results_preprocessd.isnull().sum()

horse_id                0
date                    0
rank                    0
prize                   0
rank_diff             260
weather               219
race_type               0
course_len              0
ground_condition        4
race_class          60543
n_horses                0
dtype: int64

In [10]:
%autoreload

In [84]:
race_info_preprocess  = preprocessing.process_race_info()

In [85]:
race_info_preprocess

,race_id,date,race_type,around,course_len,weather,ground_state,race_class,place
8,202301010101,2023-07-22,1,0.0,1200,0,0.0,1,1
9,202301010102,2023-07-22,0,0.0,1000,0,0.0,1,1
10,202301010103,2023-07-22,0,0.0,1700,0,0.0,1,1
11,202301010104,2023-07-22,1,0.0,1500,0,0.0,1,1
12,202301010105,2023-07-22,0,0.0,1700,0,0.0,0,1
...,...,...,...,...,...,...,...,...,...
3459,202310030808,2023-09-03,0,0.0,1000,0,0.0,2,10
3460,202310030809,2023-09-03,1,0.0,1200,0,0.0,3,10
3461,202310030810,2023-09-03,0,0.0,1700,0,0.0,4,10
3462,202310030811,2023-09-03,1,0.0,1200,0,0.0,6,10


# 特徴量作成

In [88]:
fp = FeatureCreator()
features = fp.create_features()

In [89]:
# 重複チェック
features.duplicated(subset=["race_id", "horse_id"]).sum()

0

# 学習

In [97]:
%autoreload

In [94]:
trainer = Trainer()

In [2]:
evaluation_df = trainer.run(test_start_date="2023-10-01")

NameError: name 'trainer' is not defined

# 精度評価

In [1]:
from evaluationing import Evaluator
%load_ext autoreload

In [2]:
evaluator = Evaluator()

In [3]:
evaluator.summarize_box_top_n(n=1, save_filename="box_summary_top1.csv")

,bet_type,bet_type,hitrate_model,hitrate_pop,returnrate_model,returnrate_pop
0,単勝,単勝,0.328704,0.331019,0.797454,0.798727
1,複勝,複勝,0.635417,0.641204,0.835069,0.838194


In [4]:
evaluator.summarize_box_top_n(n=2, save_filename="box_summary_top2.csv")


,bet_type,bet_type,hitrate_model,hitrate_pop,returnrate_model,returnrate_pop
0,ワイド,ワイド,0.340278,0.324074,0.892593,0.809606
1,単勝,単勝,0.548611,0.543981,0.862674,0.836285
2,複勝,複勝,0.841435,0.842593,0.862095,0.839410
3,馬単,馬単,0.179398,0.170139,0.942824,0.811748
4,馬連,馬連,0.179398,0.170139,0.962153,0.829977


In [5]:
evaluator.summarize_box_top_n(n=3, save_filename="box_summary_top3.csv")


,bet_type,bet_type,hitrate_model,hitrate_pop,returnrate_model,returnrate_pop
0,ワイド,ワイド,0.563657,0.559028,0.813927,0.802353
1,三連単,三連単,0.093750,0.093750,0.702894,0.607870
2,三連複,三連複,0.093750,0.093750,0.765972,0.731713
3,単勝,単勝,0.675926,0.663194,0.836381,0.794676
4,複勝,複勝,0.930556,0.924769,0.836844,0.824807
5,馬単,馬単,0.336806,0.322917,0.814275,0.713117
6,馬連,馬連,0.336806,0.322917,0.829861,0.733719
